In [1]:
def solve(lines):
    grid = lines
    rows = len(grid)
    cols = len(grid[0]) if rows > 0 else 0

    freq_map = {}
    for r in range(rows):
        for c in range(cols):
            ch = grid[r][c]
            if ch != '.':
                freq_map.setdefault(ch, []).append((r, c))

    antinodes = set()

    for freq, antenna_list in freq_map.items():
        n = len(antenna_list)
        if n < 2:
            continue
        for i in range(n):
            for j in range(i+1, n):
                rA, cA = antenna_list[i]
                rB, cB = antenna_list[j]

                r1 = 2*rA - rB
                c1 = 2*cA - cB
                r2 = 2*rB - rA
                c2 = 2*cB - cA

                if 0 <= r1 < rows and 0 <= c1 < cols:
                    antinodes.add((r1, c1))
                if 0 <= r2 < rows and 0 <= c2 < cols:
                    antinodes.add((r2, c2))

    return len(antinodes)


In [2]:
lines = [
"............0.......................Bn...........v",
".........0.....................8.........P.....D..",
"........M...........Q..0..8...h.......P...........",
"......M.A......c...................n..............",
".....C..................A.........................",
".M.AC....................................v........",
"........C..........W...w....J........Q...........y",
".....i..............0.....nW.......w.Zv...6.......",
"........c....................A.........Pm........D",
".............t.........x..........P....y....m.....",
"........................w...x.......F....Z........",
"...............Q......x6.......S......Z..O.......J",
"..............o.u........x....6.....r.D..M........",
"............c...o........u...Y....................",
".........i............9..............g............",
".....................d..WC..8.........J.g.........",
"...........X.c...............d...........m........",
"....................9.dR...........m......y.......",
".............o.....9.......Y.6.OS...n..........F..",
"......i..................a..Q...r.Y.............U.",
".....N......X......u..Ot...a......j......7........",
"..........q..X......t.....uH.......j.r..S.7.......",
"..........l...t....K.......................J......",
"...............9..............OB..................",
"...l.R...q..............g.......Y.7..V.......S....",
"..........................a.D............V........",
"......R.5...v.....W.............KB............U...",
"........Kp..F.N...........2.....B..............U..",
"..............................d..........h........",
"...L...NX...l...R...w..........F...........7......",
"..q.L......5.........................j............",
".q.............5.......g..4.......................",
"............p...................s2..............Z.",
"......L...p...........................s..I........",
"........N..............................H..........",
"............5......................2.......hV.....",
".............3..........1.......f.a...V...........",
".....K..................................Hz....j...",
".............k.b..G................I.....U........",
".............1......................h.............",
"...........p...........L.....s....4T..............",
".b..................G....s.T......I...............",
"............................H...........T4........",
"...............lk.................T...............",
"..i........................1........Iz............",
"..............b...........1........G..............",
"....b..............G..............................",
"........3......k............f..............4......",
"3.............k.2.....................z...........",
"...........3......................z..f............"
]

print(solve(lines))


426


In [3]:
def solve(lines):
    from math import gcd, floor, ceil
    grid = lines
    rows = len(grid)
    cols = len(grid[0]) if rows > 0 else 0

    # Agrupar antenas por frequência
    freq_map = {}
    for r in range(rows):
        for c in range(cols):
            ch = grid[r][c]
            if ch != '.':
                freq_map.setdefault(ch, []).append((r, c))

    antinodes = set()
    processed_lines = set()  # Para evitar processamento duplicado da mesma linha

    for freq, antenna_list in freq_map.items():
        if len(antenna_list) < 2:
            # Apenas uma antena dessa frequência, sem linhas
            continue

        # Cada par de antenas define uma linha
        n = len(antenna_list)
        for i in range(n):
            for j in range(i+1, n):
                rA, cA = antenna_list[i]
                rB, cB = antenna_list[j]

                dr = rB - rA
                dc = cB - cA
                g = gcd(dr, dc)
                drp = dr // g
                dcp = dc // g

                # Normalizar direção para ter uma representação única
                if drp < 0:
                    drp = -drp
                    dcp = -dcp
                elif drp == 0 and dcp < 0:
                    dcp = -dcp

                # Cálculo do offset da linha usando a normal perpendicular (dcp, -drp)
                # offset = r*dcp - c*drp é constante para todos pontos na linha
                offset = rA*dcp - cA*drp

                line_id = (freq, drp, dcp, offset)
                if line_id in processed_lines:
                    continue
                processed_lines.add(line_id)

                # Agora enumerar todos os pontos da linha dentro do mapa
                # Precisamos encontrar todos os (r,c) tais que:
                # r = r0 + k*drp
                # c = c0 + k*dcp
                # para algum ponto base (r0,c0) na linha. Podemos usar (rA,cA).
                r0, c0 = rA, cA

                # Se a linha é horizontal (drp=0), então r=r0 constante.
                # Precisamos varrer c=c0+k*dcp dentro [0, cols-1].
                if drp == 0:
                    # Verifica se r0 está dentro do mapa
                    if 0 <= r0 < rows:
                        # Encontrar intervalos para c
                        # 0 <= c0 + k*dcp < cols
                        # (0 - c0)/dcp <= k < (cols - 1 - c0)/dcp
                        # Considerar dcp positivo ou negativo
                        if dcp > 0:
                            kmin = ceil((0 - c0)/dcp)
                            kmax = floor((cols - 1 - c0)/dcp)
                        else:
                            kmin = ceil(((cols - 1) - c0)/dcp)
                            kmax = floor((0 - c0)/dcp)

                        for k in range(kmin, kmax+1):
                            rr = r0
                            cc = c0 + k*dcp
                            antinodes.add((rr, cc))

                # Se a linha é vertical (dcp=0), então c=c0 constante.
                # Precisamos varrer r=r0+k*drp dentro [0, rows-1].
                elif dcp == 0:
                    # Verifica se c0 está dentro do mapa
                    if 0 <= c0 < cols:
                        # 0 <= r0 + k*drp < rows
                        if drp > 0:
                            kmin = ceil((0 - r0)/drp)
                            kmax = floor((rows - 1 - r0)/drp)
                        else:
                            kmin = ceil(((rows - 1) - r0)/drp)
                            kmax = floor((0 - r0)/drp)

                        for k in range(kmin, kmax+1):
                            rr = r0 + k*drp
                            cc = c0
                            antinodes.add((rr, cc))

                else:
                    # Linha oblíqua
                    # Precisamos que 0 <= r0+k*drp < rows e 0 <= c0+k*dcp < cols.
                    # Encontrar intervalo para k:
                    # Para r: 0 <= r0 + k*drp < rows
                    # (0 - r0)/drp <= k < (rows-1 - r0)/drp, considerar sinal de drp
                    if drp > 0:
                        kr_min = ceil((0 - r0)/drp)
                        kr_max = floor((rows - 1 - r0)/drp)
                    else:
                        # drp < 0
                        kr_min = ceil(((rows - 1) - r0)/drp)
                        kr_max = floor((0 - r0)/drp)

                    # Para c: 0 <= c0 + k*dcp < cols
                    # (0 - c0)/dcp <= k < (cols - 1 - c0)/dcp
                    if dcp > 0:
                        kc_min = ceil((0 - c0)/dcp)
                        kc_max = floor((cols - 1 - c0)/dcp)
                    else:
                        # dcp < 0
                        kc_min = ceil(((cols - 1) - c0)/dcp)
                        kc_max = floor((0 - c0)/dcp)

                    # k deve estar no intervalo [max(kr_min, kc_min), min(kr_max, kc_max)]
                    kstart = max(kr_min, kc_min)
                    kend = min(kr_max, kc_max)

                    for k in range(kstart, kend+1):
                        rr = r0 + k*drp
                        cc = c0 + k*dcp
                        antinodes.add((rr, cc))

    return len(antinodes)

# Exemplo de uso no Jupyter:
# lines = [...] # Coloque aqui o seu mapa ou leia de um arquivo
# print(solve(lines))
